In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup


import pymongo
import dns
import json

import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [6]:
def insert_into_db(collection_name,object_to_add):
    client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
    db = client.stats
    db[collection_name].insert_one(object_to_add)

In [27]:
def scrape_stats(start_year : int, end_year : int):
    
#     Initialize mongodb connection
    client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
    db = client.stats

    
    for year in range(start_year, end_year + 1):
        year = str(year)
        
        
        url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        
        
        for i, row in enumerate(soup.find("div", id = "div_per_game_stats").find('tbody').find_all('tr')):
            
            stat_dictionary = {}
            
            if "thead" in row.get('class'):                
                continue
            
            try:
                for ind, col in enumerate(row):
                    if ind != 0:
                        stat_dictionary[col.get('data-stat')] = col.text
                    
            except AttributeError:
                pass
            
            
            
            advanced_stats_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
            advanced_source = requests.get(advanced_stats_url).text
            soup_advanced = BeautifulSoup(advanced_source, 'lxml')
            
            
            advanced_col = soup_advanced.find("div", id = "div_advanced_stats").find('tbody').find_all('tr')[i]
            
            try:
                for ind, col in enumerate(advanced_col):
                    if ind != 0 and ind != 1:
                        stat_dictionary[col.get('data-stat')] = col.text
            
            except AttributeError:
                pass
            
            
            db[year].insert_one(stat_dictionary)
#             insert_into_db(year, stat_dictionary)
            
            
        
            
    


In [30]:
scrape_stats(2019,2022)

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))